# 1 in 0 is T01

In [19]:
import numpy as np
import scipy.linalg 
import math

def fromRPcreateHomogenous(R, p):
    res = np.c_[R,p]
    if len(R) == 3: 
        res = np.r_[res, [[0,0,0,1]]]
    if len(R) == 2:
        res = np.r_[res, [[0,0,1]]]
    return res

def formSkewSymm(a):
    res = np.zeros([3,3])
    res[1,0] = a[2,0]
    res[2,0] = -a[1,0]
    res[2,1] = a[0,0]
    
    res[0,1] = -a[2,0]
    res[0,2] = a[1,0]
    res[1,2] = -a[0,0]
    return res

def vectorFromSkewSymm(R):
    retVal = []
    retVal.append([R[2][1]])
    retVal.append([-R[2][0]])
    retVal.append([R[1][0]])
    return np.array(retVal)

def fromHomogenouscreateRP(H):
    R = np.array([[H[i][j] for j in range(3)] for i in range(3)])
    p = np.array([[H[i][j] for j in range(3,4)] for i in range(3)])
    return R, p

def decomposeOmega(w01, t):
    w01 = w01 * t
    theta = ((w01**2).sum()**0.5)
    w01_hat = w01 / theta
    return w01_hat, theta

def xRotation(theta):
    rotationMatrix = np.zeros([3,3])
    rotationMatrix[0,0] = 1
    rotationMatrix[1,0] = 0
    rotationMatrix[2,0] = 0

    rotationMatrix[0,1] = 0
    rotationMatrix[1,1] = np.cos(theta)
    rotationMatrix[2,1] = np.sin(theta)
    
    rotationMatrix[0,2] = 0
    rotationMatrix[1,2] = -np.sin(theta)
    rotationMatrix[2,2] = np.cos(theta)
    
    return rotationMatrix

def yRotation(theta):
    rotationMatrix = np.zeros([3,3])
    rotationMatrix[0,0] = np.cos(theta)
    rotationMatrix[1,0] = 0
    rotationMatrix[2,0] = -np.sin(theta)

    rotationMatrix[0,1] = 0
    rotationMatrix[1,1] = 1
    rotationMatrix[2,1] = 0
    
    rotationMatrix[0,2] = np.sin(theta)
    rotationMatrix[1,2] = 0
    rotationMatrix[2,2] = np.cos(theta)
    
    return rotationMatrix

def zRotation(theta):
    rotationMatrix = np.zeros([3,3])
    rotationMatrix[0,0] = np.cos(theta)
    rotationMatrix[1,0] = np.sin(theta)
    rotationMatrix[2,0] = 0

    rotationMatrix[0,1] = -np.sin(theta)
    rotationMatrix[1,1] = np.cos(theta)
    rotationMatrix[2,1] = 0
    
    rotationMatrix[0,2] = 0
    rotationMatrix[1,2] = 0
    rotationMatrix[2,2] = 1
    
    return rotationMatrix

def prismatic(omegaHat, theta):
    # for prismatic joint
    M = np.eye(4)
    M[0,3] = theta*omegaHat[0,0]
    M[1,3] = theta*omegaHat[1,0]
    M[2,3] = theta*omegaHat[2,0]
    return M
    
def revolute(omegaHat, theta):
    # for revolute joint
    M = np.zeros([4,4])
    M[0:3,0:3] = scipy.linalg.expm(formSkewSymm(theta*omegaHat))
    M[3,3] = 1
    return M

def translation(a):
    M = np.eye(4)
    M[0,3] = a[0]
    M[1,3] = a[1]
    M[2,3] = a[2]
    return M


In [2]:
R1 = np.array([[-0.62622759, -0.16235431, 0.64264942], [-0.44090625, -0.75360797, -0.61861169], [0.55851369, -0.76861652, 0.19149539]])
R2 = np.array([[0.46774986, -0.39274677, -0.89155545], [0.89953316, 0.49067498, 0.17636038], [0.40008193, -0.87048761, 0.37278598]])
R3 = np.array([[-0.62213980, -0.61043344, 0.49021739], [0.72932799, -0.67953959, 0.07941431], [0.28464497, 0.40693606, 0.86797482]])
R4 = np.array([[0.96332340, 0.02913225, 0.26675707], [0.01344178, 0.98760288, -0.15639652], [-0.26800623, 0.15424611, 0.95098938]])
R5 = np.array([[-0.37637574, -0.66115723, -0.96374834], [0.14339310, -0.79590400, 0.27063580], [-1.11470268, -0.11208157, 0.42831922]])

lst = [R1,R2,R3,R4,R5]
i = 0
for t in lst:
    i += 1
    inv = np.linalg.inv(t)
    trans = np.transpose(t)
#     print(inv)
#     print(trans)
#     print("\n\n")

In [3]:
theta1 = 0.88
theta2 = -1.13
theta3 = -1.36

ans = np.matmul(np.matmul(xRotation(theta1), yRotation(theta2)), xRotation(theta3))
print(np.array_repr(ans))

array([[ 0.42665981,  0.88439267, -0.189238  ],
       [-0.69706564,  0.45488088,  0.55424082],
       [ 0.57624726, -0.10456097,  0.81055913]])


In [9]:
# alpha = atan2(r21, r11)  
# beta = atan2(-r31, sqrt(r11 sq+ r21 sq))
# gamma = atan2(r32,r33)

# roll pitch yaw in the opp dir
R = np.array([[0.516342801446, 0.036819277972, 0.855590119254], [-0.022647086399, 0.999312993234, -0.029336854504], [-0.856082484853, -0.004228749713, 0.516821919817]])

print(math.atan2(R[2,1], R[2,2])) #roll
print(math.atan2(-R[2,0], ((R[0,0])**2 + (R[1,0])**2)**0.5))  #pitch
print(math.atan2(R[1,0], R[0,0])) #yaw

-0.008182035600445007
1.0276416531540427
-0.04383247067274513


In [11]:
# robot real time calc stuff
import math
# alpha = atan2(r21, r11)
# beta = atan2(-r31, sqrt(r11 sq+ r21 sq))
# gamma = atan2(r32,r33)


print(math.degrees(math.atan2(0.2076, 0.1593))) #z rotation angle
print(math.degrees(math.atan2(-0.9651, ((0.1593)**2 + (0.2067)**2)**0.5)))  #y rotation angle
print(math.degrees(math.atan2(0.2615, 0.0120))) #x rotation angle

# do in order of z y and x to get right ans. do not forget the - in y angle

52.49949323562463
-74.86910073344468
87.37259140724802


In [16]:
# second thing should match
# add 1 for p and 0 for q IMPPPPPPPP
T_07 = np.array([[-0.08145638, 0.22324853, -0.97135212, 0.48046534], [0.69118901, -0.68950319, -0.21643267, 0.75100898], [-0.71806867, -0.68901773, -0.09814250, 0.76200630], [0.00000000, 0.00000000, 0.00000000, 1.00000000]])
p_0 = np.array([[0.75212753], [0.36966967], [0.24488195], [1]])

T_60 = np.array([[-0.33649944, 0.21871368, -0.91593256, 0.35115966], [-0.30398643, 0.89535217, 0.32547924, -0.41942768], [0.89126897, 0.38795465, -0.23479952, -0.18010021], [0.00000000, 0.00000000, 0.00000000, 1.00000000]])
v_0 = np.array([[-0.68157552], [0.94047922], [-0.04775126], [0]])

ans = np.linalg.inv(T_07)@p_0
ans = T_60 @ v_0

print(np.array_repr(ans[:3]))

array([[ 0.47878239],
       [ 1.03370778],
       [-0.23139185]])


In [18]:

p_5in8 = np.array([[-0.89884572], [0.11213099], [0.18555571]])
R_5in8 = np.array([[0.16706564, -0.18541256, -0.96835492], [0.63843980, -0.72809102, 0.24955578], [-0.75132129, -0.65992852, -0.00326424]])

H = fromRPcreateHomogenous(R_5in8, p_5in8)
R, p = fromHomogenouscreateRP(np.linalg.inv(H))
print(R)
print(p)

[[ 0.16706564  0.6384398  -0.7513213 ]
 [-0.18541255 -0.72809102 -0.65992852]
 [-0.96835492  0.24955579 -0.00326423]]
[[ 0.21798931]
 [ 0.03743779]
 [-0.89777892]]


In [5]:

w01 = np.array([[2.33], [-1.61], [-0.96]])
t = 10.18

w = formSkewSymm(w01*t)
R = scipy.linalg.expm(w)
print(np.array_repr(R))


array([[ 0.82807245, -0.44899099,  0.33571283],
       [ 0.08188406,  0.68926605,  0.71986618],
       [-0.55460888, -0.56861182,  0.60752744]])


$\dot{R}^0_1 = R_{01}.[w^1_{01}]$

$[w_S] = \dot{R_{SB}} . R^T_{SB}$

$[w_B] =  R^T_{SB} . \dot{R_{SB}}$

In [6]:

R_1in0 = np.array([[0.06517374, 0.21102116, 0.97530634], [-0.92713400, -0.34863640, 0.13738703], [0.36901886, -0.91319370, 0.17292295]])
w_01in1 = np.array([[-0.85642499], [-0.83602923], [-0.49611978]])

W = formSkewSymm(w_01in1)
print(np.array_repr(np.matmul(R_1in0, W)))

array([[ 0.71069284, -0.80294274,  0.12623664],
       [ 0.28782499, -0.5776312 ,  0.4765302 ],
       [ 0.5976221 ,  0.03498202, -1.09059246]])


find $w^1_{0,1}$ when $R^0_1$ and $w^0_{1,0}$ are given

In [7]:

R_1in0 = np.array([[-0.66647761, -0.44255665, 0.59995934], [-0.74241324, 0.32053063, -0.58828794], [0.06804539, -0.83749849, -0.54218640]])
w_10in0 = np.array([[-0.33700861], [-0.64989473], [-1.04311870]])

RDot = np.matmul( formSkewSymm(w_10in0), R_1in0) 

ans = np.matmul(np.transpose(R_1in0), RDot)

ans = -vectorFromSkewSymm(ans)
print(np.array_repr(ans ) )

array([[-0.63611972],
       [-0.81444458],
       [-0.74569854]])


find $w^0_{1,0}$ when $R^0_1$ and $w^1_{0,1}$ are given

In [8]:
R_1in0 = np.array([[-0.00225220, 0.02092339, -0.99977855], [-0.99395831, 0.10966456, 0.00453415], [0.10973515, 0.99374841, 0.02054999]])
w_01in1 = np.array([[-0.76918454], [0.36172784], [-0.17436341]])

RDot = np.matmul(R_1in0, formSkewSymm(w_01in1)) 

ans = np.matmul(RDot, np.transpose(R_1in0))
ans = -vectorFromSkewSymm(ans)  #we need - coz 01 and 10 diff
print(np.array_repr(ans ) )

array([[-0.18362573],
       [-0.80341551],
       [-0.27147672]])


In [21]:
T = np.zeros([4,4])
T[3,3] = 1

# x y and z pos
T[0,3] = 0
T[1,3] = 0
T[2,3] = 2

# new x in s frame
T[0,0] = -1
T[1,0] = 0
T[2,0] = 0
# new y in s frame
T[0,1] = 0
T[1,1] = 0
T[2,1] = 1
# new z in s frame
T[0,2] = 0
T[1,2] = 1
T[2,2] = 0

theta = np.array([[-0.72], [0.07000000], [-0.68000000]])


M1 = translation([0, 2, 0])

omegaHat = np.array([[0],[1],[0]])
M2 = revolute(omegaHat, theta[0,0])

M3 = translation([2, -2, 0])

omegaHat = np.array([[0],[0],[1]])
M4 = revolute(omegaHat, theta[1,0])

M5 = translation([0, -2, 0]) 

omegaHat = np.array([[1],[0],[0]])
M6 = revolute(omegaHat, theta[2,0])

ans = M1.dot(M2.dot(M3.dot(M4.dot(M5.dot(M6.dot(T))))))

print(np.array_repr(ans))

array([[-0.75180573, -0.65938467,  0.        , -1.31876934],
       [ 0.        ,  0.        ,  1.        ,  2.        ],
       [-0.65938467,  0.75180573,  0.        ,  1.50361146],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])


In [ ]:
def to_skew(w):
    return np.array([[0, -w[2], w[1]],[w[2],0,-w[0]],[-w[1], w[0],0]])
def adjoint(T):
    ad = np.zeros([6,6])
    ad[0:3,0:3] = T[0:3,0:3]
    ad[3:6,3:6] = T[0:3,0:3]
    ad[3:6,0:3] = to_skew(T[0:3,3]).dot(T[0:3,0:3])
    return ad



T = np.eye(4)
T[3,3] = 1

# x y and z pos
T[0,3] = 0
T[1,3] = 2
T[2,3] = 0


theta = -0.83
dtheta = -0.33 

S_2 = np.array([[0], [0], [1], [0], [0], [0]])

M1 = translation([0, 0, -2])

omegaHat = np.array([[0],[-1],[0]])
M2 = revolute(omegaHat, theta)


ad = adjoint(np.linalg.inv(T))    #for body

# ad = adjoint(M1 @M2)            #for space

ans = np.matmul(ad, S_2) * dtheta

print(np.array_repr(ans))

In [ ]:
theta = np.array([[-0.44000000], [0.48000000], [-0.36000000], [0.04000000], [0.64000000], [0.75000000], [0.60000000], [-0.96000000]])
dtheta = np.array([[0.14000000], [-0.15000000], [0.16000000], [-0.76000000], [-0.74000000], [-0.95000000], [-0.62000000], [-0.81000000]])
jacobian = np.array([[0.00000000, 0.81823917, 0.00000000, -0.41314261, -0.41314261,  0.82533561, 0.00000000, 1.00000000], [0.00000000, 0.39380403, 0.00000000, -0.10376723, -0.10376723,  -0.46255035, 0.57351999, -0.00000000], [0.00000000, 0.41881147, 0.00000000, 0.90473507, 0.90473507, 0.32383374,  0.81919157, 0.00000000], [0.81823917, 4.75655259, 0.81823917, 1.09207376, 2.69527184, -0.92510071,  3.14703997, -0.00000000], [0.39380403, -9.13611339, 0.39380403, 5.77162077, 6.63164482,  -1.94969668, 1.63838314, -2.00000000], [0.41881147, -0.70236715, 0.41881147, 1.16065723, 1.99138855,  -0.42711525, -1.14703997, -0.00000000]])
T_1in0 = np.array([[-0.55105034, 0.32971048, 0.76657323, 6.27621362], [0.16379310, -0.85802645, 0.48678787, 4.17178808], [0.81823917, 0.39380403, 0.41881147, 3.94448633], [0.00000000, 0.00000000, 0.00000000, 1.00000000]])

# space to body
ans = adjoint(np.linalg.inv(T_1in0)).dot(jacobian.dot(dtheta))

# body to space
ans = adjoint(T_1in0).dot(jacobian.dot(dtheta))

# same to same
# ans = jacobian.dot(dtheta)

print(np.array_repr(ans))

In [ ]:
T = np.zeros([4,4])
T[3,3] = 1

# x y and z pos
T[0,3] = 0
T[1,3] = 0
T[2,3] = -2

# new x in s frame
T[0,0] = 0
T[1,0] = 0
T[2,0] = -1
# new y in s frame
T[0,1] = 0
T[1,1] = -1
T[2,1] = 0
# new z in s frame
T[0,2] = -1
T[1,2] = 0
T[2,2] = 0

theta = np.array([[0.14000000], [-0.97000000], [0.88000000]])

M1 = translation([0, 0, -2])

S1 = np.array([[0],[1],[0],[0],[0],[0]])
omegaHat = np.array([[0],[1],[0]])
M2 = revolute(omegaHat, theta[0])
t0a = M1@M2

M3 = translation([0, 2, -2])

S2 = np.array([[0],[0],[0],[0],[1],[0]])
omegaHat = np.array([[0],[1],[0]])
M4 = prismatic(omegaHat, theta[1])
tab = M3@M4

M5 = translation([0, 2, 0]) 

S3 = np.array([[-1],[0],[0],[0],[0],[0]])
omegaHat = np.array([[-1],[0],[0]])
M6 = revolute(omegaHat, theta[2])
tbc = M5@M6

# print(t0a, tab, tbc, sep="\n")

# space J
# S1 = adjoint(t0a).dot(S1)
# S2 = adjoint(t0a@tab).dot(S2)
# S3 = adjoint(t0a@tab@tbc).dot(S3)

# body J
S1 = adjoint(np.linalg.inv(tab@tbc@T)).dot(S1)
S2 = adjoint(np.linalg.inv(tbc@T)).dot(S2)
S3 = adjoint(np.linalg.inv(T)).dot(S3)

J = np.c_[S1, S2]
J = np.c_[J, S3]

print(np.array_repr(J))